In [ ]:
!pip install wrds

import wrds
import pandas as pd

# Connect to WRDS - will prompt for your WRDS username and password
db = wrds.Connection()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.3 MB/s eta 0:00:00
Enter your WRDS username [root]:yy3216
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
sp500 = pd.read_csv('/content/sp500_with_gvkey_permno.csv')
sp500

,ticker,name,gvkey,permno,comnam
0,A,Agilent Technologies,126554.0,87432.0,AGILENT TECHNOLOGIES INC
1,AAPL,Apple Inc.,1690.0,14593.0,APPLE INC
2,ABBV,AbbVie,16101.0,13721.0,ABBVIE INC
3,ABNB,Airbnb,37460.0,20190.0,AIRBNB INC
4,ABT,Abbott Laboratories,1078.0,20482.0,ABBOTT LABORATORIES
...,...,...,...,...,...
498,XYL,Xylem Inc.,189491.0,13035.0,XYLEM INC
499,YUM,Yum! Brands,65417.0,85348.0,YUM BRANDS INC
500,ZBH,Zimmer Biomet,144559.0,89070.0,ZIMMER BIOMET HOLDINGS INC
501,ZBRA,Zebra Technologies,24405.0,76795.0,ZEBRA TECHNOLOGIES CORP


In [ ]:
gvkeys = sp500['gvkey'].tolist()
permnos = sp500['permno'].tolist()
tickers = sp500['ticker'].tolist()

In [ ]:
# Convert the list of tickers into a format suitable for SQL IN clause
tickers_str = ','.join(f"'{ticker}'" for ticker in tickers)

# Retrieve earnings estimates from IBES
ibes_estimates = db.raw_sql(f"""
    SELECT oftic AS ticker, cname, fpedats, curr_act, actual
    FROM tr_ibes.statsum_epsus
    WHERE fpedats BETWEEN '2021-01-01' AND '2024-12-31'
      AND ticker IN ({tickers_str})
""")

# Remove duplicate rows
ibes_estimates = ibes_estimates.drop_duplicates()

# Save the data to a CSV file
ibes_estimates.to_csv("sp500_earnings_estimates_2021_2024.csv", index=False)
print("Saved: Earnings estimates data")

Saved: Earnings estimates data


In [ ]:
# Break permno list into chunks to avoid SQL limits
# Convert permnos to integers to avoid issues with NaN
permnos = [int(p) for p in sp500['permno'] if pd.notna(p)]
permno_chunks = [permnos[i:i + 500] for i in range(0, len(permnos), 500)]

corp_action = pd.DataFrame()
for chunk in permno_chunks:
    permno_str = ','.join(str(p) for p in chunk)
    temp = db.raw_sql(f"""
        SELECT permno, distcd, dclrdt, rcrddt, paydt, facpr, facshr, divamt
        FROM crsp_a_stock.dsedist
        WHERE permno IN ({permno_str})
          AND dclrdt BETWEEN '2021-01-01' AND '2024-12-31'
          AND distcd BETWEEN 1000 AND 6999
    """)
    corp_action = pd.concat([corp_action, temp])

corp_action = corp_action.merge(sp500[['permno', 'ticker']], on='permno', how='left')

def classify_distcd(code):
    if 1000 <= code < 2000:
        return 'Dividend'
    elif 2000 <= code < 3000:
        return 'Liquidation'
    elif 3000 <= code < 4000:
        return 'Acquisition / Reorganization'
    elif 4000 <= code < 5000:
        return 'Rights'
    elif 5000 <= code < 6000:
        return 'Stock'
    elif 6000 <= code < 7000:
        return 'Offer / Issuances'
    else:
        return 'Other'

corp_action['dist_type'] = corp_action['distcd'].apply(classify_distcd)


distcd_mapping = {
    1232: 'Regular Cash Dividend',
    1242: 'Special Cash Dividend',
    1270: 'Liquidating Cash Dividend',
    3252: 'Merger / Acquisition',
    5523: 'Stock Split',
    5530: 'Stock Dividend',
    5570: 'Reverse Stock Split',
    4260: 'Rights Offering',
    6270: 'Public Offering',
}

corp_action['dist_detail'] = corp_action['distcd'].map(distcd_mapping).fillna('Other')


corp_action.to_csv("sp500_corporate_action_2021_2024.csv", index=False)
print("Saved: Corporate Action data")

Saved: Corporate Action data


In [ ]:
# Break gvkey list into chunks to avoid SQL limits
gvkey_chunks = [gvkeys[i:i + 500] for i in range(0, len(gvkeys), 500)]

executive = pd.DataFrame()
for chunk in gvkey_chunks:
    gvkey_str = ','.join(f"'{g}'" for g in chunk)
    temp = db.raw_sql(f"""
       SELECT coname, execid, gvkey, inddesc, spcode, ticker, year, exec_fullname, gender, becameceo, leftco, leftofc, reason, rejoin, releft, salary, shrown_tot
        FROM 	comp_execucomp.anncomp
        WHERE gvkey IN ({gvkey_str})
          AND becameceo BETWEEN '2021-01-01' AND '2024-12-31'
          OR leftco BETWEEN '2021-01-01' AND '2024-12-31'
          OR releft BETWEEN '2021-01-01' AND '2024-12-31'
          OR rejoin BETWEEN '2021-01-01' AND '2024-12-31'
          OR leftofc BETWEEN '2021-01-01' AND '2024-12-31'

    """)
    executive = pd.concat([executive, temp])

executive.to_csv("sp500_executive_2021_2024.csv", index=False)
print("Saved: Executive data")

Saved: Executive data


In [ ]:
tickers = sp500['ticker'].dropna().unique().tolist()
ticker_str = ",".join(f"'{t}'" for t in tickers)
ticker_sql = f"({ticker_str})"

In [ ]:
ticker_sql

"('A','AAPL','ABBV','ABNB','ABT','ACGL','ACN','ADBE','ADI','ADM','ADP','ADSK','AEE','AEP','AES','AFL','AIG','AIZ','AJG','AKAM','ALB','ALGN','ALL','ALLE','AMAT','AMCR','AMD','AME','AMGN','AMP','AMT','AMZN','ANET','ANSS','AON','AOS','APA','APD','APH','APO','APTV','ARE','ATO','AVB','AVGO','AVY','AWK','AXON','AXP','AZO','BA','BAC','BALL','BAX','BBY','BDX','BEN','BF-B','BG','BIIB','BK','BKNG','BKR','BLDR','BLK','BMY','BR','BRK-B','BRO','BSX','BX','BXP','C','CAG','CAH','CARR','CAT','CB','CBOE','CBRE','CCI','CCL','CDNS','CDW','CEG','CF','CFG','CHD','CHRW','CHTR','CI','CINF','CL','CLX','CMCSA','CME','CMG','CMI','CMS','CNC','CNP','COF','COO','COP','COR','COST','CPAY','CPB','CPRT','CPT','CRL','CRM','CRWD','CSCO','CSGP','CSX','CTAS','CTRA','CTSH','CTVA','CVS','CVX','CZR','D','DAL','DASH','DAY','DD','DE','DECK','DELL','DFS','DG','DGX','DHI','DHR','DIS','DLR','DLTR','DOC','DOV','DOW','DPZ','DRI','DTE','DUK','DVA','DVN','DXCM','EA','EBAY','ECL','ED','EFX','EG','EIX','EL','ELV','EMN','EMR','ENPH','EO

In [ ]:
num_ticker = ticker_sql.count(",") + 1
print(f"Number of ticker in SQL string: {num_ticker}")

Number of ticker in SQL string: 503


In [ ]:
query = f"""
SELECT GVKEY AS gvkey, PERMNO, TICKER, public_date,
       bm, divyield, pe_exi, pe_inc, capei,
       gprof, gpm, npm, roa, roe, capital_ratio,
       de_ratio, quick_ratio, at_turn, inv_turn
FROM wrdsapps_finratio.firm_ratio
WHERE public_date BETWEEN '2021-01-01' AND '2025-01-01'
  AND TICKER IN {ticker_sql}
"""
fin_ratio_df = db.raw_sql(query, date_cols=['public_date'])

In [ ]:
fin_ratio_df

,gvkey,permno,ticker,public_date,bm,divyield,pe_exi,pe_inc,capei,gprof,gpm,npm,roa,roe,capital_ratio,de_ratio,quick_ratio,at_turn,inv_turn
0,001988,57568,BALL,2022-08-31,0.188015,0.014334,24.47807,24.47807,30.070239,0.136307,0.184652,0.049612,0.108461,0.170445,0.69812,4.543215,0.627071,0.738182,5.975453
1,001300,10145,HON,2021-01-31,0.16924,0.019041,28.030129,28.030129,28.329023,0.198829,0.363705,0.149911,0.124399,0.253663,0.454253,2.347821,1.213914,0.546676,4.581271
2,001300,10145,HON,2021-02-28,0.132664,0.018384,30.111607,30.111607,29.136398,0.182362,0.360879,0.146429,0.118006,0.239994,0.491805,2.666192,1.233839,0.529542,4.682155
3,001300,10145,HON,2021-03-31,0.132664,0.017137,32.302083,32.302083,31.255933,0.182362,0.360879,0.146429,0.118006,0.239994,0.491805,2.666192,1.233839,0.529542,4.682155
4,001300,10145,HON,2021-04-30,0.132664,0.016679,33.190476,33.190476,32.071915,0.182362,0.360879,0.146429,0.118006,0.239994,0.491805,2.666192,1.233839,0.529542,4.682155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20544,186310,12623,HII,2024-08-31,0.467277,0.01839,15.001061,15.001061,18.105682,0.184776,0.173843,0.063524,0.09083,0.175835,0.36466,1.747241,0.844256,1.062894,50.079794
20545,186310,12623,HII,2024-09-30,0.467277,0.019669,14.025464,14.025464,16.928177,0.184776,0.173843,0.063524,0.09083,0.175835,0.36466,1.747241,0.844256,1.062894,50.079794
20546,186310,12623,HII,2024-10-31,0.467277,0.028114,9.812202,9.812202,11.842937,0.184776,0.173843,0.063524,0.09083,0.175835,0.36466,1.747241,0.844256,1.062894,50.079794
20547,186310,12623,HII,2024-11-30,0.436833,0.027284,11.175607,11.175607,12.553755,0.174208,0.166553,0.059874,0.08124,0.159954,0.345297,1.697229,0.812641,1.045964,49.532995


In [ ]:
# 用 ticker 作为键进行合并，把 comnam 加入进去
fin_ratio_with_comnam = pd.merge(fin_ratio_df, sp500[['ticker', 'comnam']], on='ticker', how='left')

# 查看结果
fin_ratio_with_comnam

,gvkey,permno,ticker,public_date,bm,divyield,pe_exi,pe_inc,capei,gprof,gpm,npm,roa,roe,capital_ratio,de_ratio,quick_ratio,at_turn,inv_turn,comnam
0,001988,57568,BALL,2022-08-31,0.188015,0.014334,24.47807,24.47807,30.070239,0.136307,0.184652,0.049612,0.108461,0.170445,0.69812,4.543215,0.627071,0.738182,5.975453,BALL CORP
1,001300,10145,HON,2021-01-31,0.16924,0.019041,28.030129,28.030129,28.329023,0.198829,0.363705,0.149911,0.124399,0.253663,0.454253,2.347821,1.213914,0.546676,4.581271,HONEYWELL INTERNATIONAL INC
2,001300,10145,HON,2021-02-28,0.132664,0.018384,30.111607,30.111607,29.136398,0.182362,0.360879,0.146429,0.118006,0.239994,0.491805,2.666192,1.233839,0.529542,4.682155,HONEYWELL INTERNATIONAL INC
3,001300,10145,HON,2021-03-31,0.132664,0.017137,32.302083,32.302083,31.255933,0.182362,0.360879,0.146429,0.118006,0.239994,0.491805,2.666192,1.233839,0.529542,4.682155,HONEYWELL INTERNATIONAL INC
4,001300,10145,HON,2021-04-30,0.132664,0.016679,33.190476,33.190476,32.071915,0.182362,0.360879,0.146429,0.118006,0.239994,0.491805,2.666192,1.233839,0.529542,4.682155,HONEYWELL INTERNATIONAL INC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20544,186310,12623,HII,2024-08-31,0.467277,0.01839,15.001061,15.001061,18.105682,0.184776,0.173843,0.063524,0.09083,0.175835,0.36466,1.747241,0.844256,1.062894,50.079794,HUNTINGTON INGALLS INDS INC
20545,186310,12623,HII,2024-09-30,0.467277,0.019669,14.025464,14.025464,16.928177,0.184776,0.173843,0.063524,0.09083,0.175835,0.36466,1.747241,0.844256,1.062894,50.079794,HUNTINGTON INGALLS INDS INC
20546,186310,12623,HII,2024-10-31,0.467277,0.028114,9.812202,9.812202,11.842937,0.184776,0.173843,0.063524,0.09083,0.175835,0.36466,1.747241,0.844256,1.062894,50.079794,HUNTINGTON INGALLS INDS INC
20547,186310,12623,HII,2024-11-30,0.436833,0.027284,11.175607,11.175607,12.553755,0.174208,0.166553,0.059874,0.08124,0.159954,0.345297,1.697229,0.812641,1.045964,49.532995,HUNTINGTON INGALLS INDS INC


In [ ]:
fin_ratio_with_comnam.to_csv("sp500_firm_ratios_2021_2024.csv", index=False)
print("✅ Export complete: sp500_firm_ratios_2021_2024.csv")

✅ Export complete: sp500_firm_ratios_2021_2024.csv


In [ ]:
from google.colab import files
files.download("sp500_earnings_estimates_2021_2024.csv")
files.download("sp500_corporate_action_2021_2024.csv")
files.download("sp500_executive_2021_2024.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>